## Procesamiento Datos Pandas

Complete el código faltante según corresponda a manera de lograr automatizar el flujo. Finalmente adjunte una imagen del envio de correo correspondiente.



El archivo candidates.csv contiene 50k filas de datos sobre candidatos. Los campos que estamos usando son:
- First Name
- Last Name
- Email
- Country
- Application Date
- Yoe (years of experience)
- Seniority
- Technology
- Code Challenge Score
- Technical Interview


## 1. Almaceno sobre DB

In [1]:
# lea el articulo:
# https://medium.com/analytics-vidhya/inserting-an-image-in-an-excel-file-using-pandas-and-sql-e88826680b05


In [2]:
!pip install xlsxwriter

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import pandas as pd
import xlsxwriter
import sqlite3
import os

DB = 'candidates.db'
TABLE_NAME = 'candidate'


In [4]:
def almacenar_pandas_to_sql(df: pd.DataFrame, database_name: str, table_name: str) -> None:
    """Procesamiento de datos de candidatos para almacenarlos en la base de datos"""

    # Renombrar columnas eliminando espacios
    column_rename = {c: c.replace(' ', '') for c in df.columns}
    df.rename(column_rename, axis='columns', inplace=True)

    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(database_name)

    # Crear o reemplazar la tabla en la base de datos
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    # Cerrar la conexión
    conn.close()

    # Imprimir información sobre el proceso
    sql_table_schema = f'{database_name}.{table_name}'
    cantidad_registros = df.shape[0]
    print(f'Se almacenaron {cantidad_registros} registros en la tabla SQL {sql_table_schema}')


In [5]:
path = './src/candidates.csv'
df = pd.read_csv(path, sep=';')

In [6]:
print(df.shape)
print(df.head(2))

(50000, 10)
   First Name  Last Name                Email Application Date Country  YOE  \
0  Bernadette  Langworth  leonard91@yahoo.com       2021-02-26  Norway    2   
1      Camryn   Reynolds  zelda56@hotmail.com       2021-09-09  Panama   10   

  Seniority     Technology  Code Challenge Score  Technical Interview Score  
0    Intern  Data Engineer                     3                          3  
1    Intern  Data Engineer                     2                         10  


In [7]:
almacenar_pandas_to_sql(df, DB, TABLE_NAME)

Se almacenaron 50000 registros en la tabla SQL candidates.db.candidate


## 2. Procesamiento Datos

Generar un reporte por separado para los paises ( 'United States of America', 'Brazil','Colombia','Ecuador') donde se deberá seleccionar a los candidatos cuyo CodeChallengeScore y TechnicalInterviewScore sea mayor igual a 7. Se deberá crear gráficos integrados sobre cada reporte y además almacenar los datos en un excel con el nombre de cada pais.

In [8]:
import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt

DB = 'candidates.db'
TABLE_NAME = 'candidate'

def genero_grafico_circular(df: pd.DataFrame, country: str) -> None:
    tec = df.groupby('Technology')['Technology'].count()
    tec.plot.pie(figsize=(11, 7))
    plt.savefig(f"./reportes/images/{country}/pie_chart.png", dpi=300, bbox_inches='tight')
    plt.close()
    print('Se generó gráfico circular...')

def genero_grafico_barras(df: pd.DataFrame, country: str) -> None:
    senority_group = df.groupby('Seniority')['Seniority'].count()
    senority_group.plot.bar()
    plt.savefig(f"./reportes/images/{country}/bar_chart.png", dpi=300, bbox_inches='tight')
    plt.close()
    print('Se generó gráfico de barras...')

# Listado de países
listado_paises = ['United States of America', 'Brazil', 'Colombia', 'Ecuador']

# 1. Lectura de datos
with sqlite3.connect(DB) as conn:
    df = pd.read_sql_query(f'select * from {TABLE_NAME}', conn)

# 2. Filtrado de datos
filterDf = df[(df['CodeChallengeScore'] >= 7) & (df['TechnicalInterviewScore'] >= 7) &
              (df['Country'].isin(listado_paises))]

# Creo el directorio para los reportes
if not os.path.isdir('./reportes'):
    os.mkdir('./reportes')
    os.mkdir('./reportes/images')

# Procesamiento por país
for country in listado_paises:
    # Creo subcarpetas
    if not os.path.isdir(f'./reportes/images/{country}'):
        os.mkdir(f'./reportes/images/{country}')

    # Filtro df
    countryDf = filterDf[filterDf['Country'] == country]

    # Genero gráficos
    genero_grafico_circular(countryDf, country)
    genero_grafico_barras(countryDf, country)

    # Genero reporte Excel por país
    with pd.ExcelWriter(f"./reportes/{country}.xlsx", engine="xlsxwriter") as excelBook:
        sheet_name = f"Report-{country}"
        countryDf.to_excel(excelBook, index=False, sheet_name=sheet_name)

        # Posiciono sobre hoja de Excel
        excel_sheet = excelBook.sheets[sheet_name]

        # Almaceno imágenes
        image_pie_path = f"./reportes/images/{country}/pie_chart.png"
        image_bar_path = f"./reportes/images/{country}/bar_chart.png"

        excel_sheet.insert_image(1, countryDf.shape[1] + 2, image_pie_path)
        excel_sheet.insert_image(countryDf.shape[0] + 2, countryDf.shape[1] + 2, image_bar_path)

        # Guardo cambios sobre Excel
        excelBook.close()

        print(f'Se generó un informe para el país {country}')

print('Se finalizó la generación de informes.')

Se generó gráfico circular...
Se generó gráfico de barras...
Se generó un informe para el país United States of America


/home/codespace/.python/current/lib/python3.10/site-packages/xlsxwriter/workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Se generó gráfico circular...
Se generó gráfico de barras...
Se generó un informe para el país Brazil


/home/codespace/.python/current/lib/python3.10/site-packages/xlsxwriter/workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Se generó gráfico circular...
Se generó gráfico de barras...
Se generó un informe para el país Colombia


/home/codespace/.python/current/lib/python3.10/site-packages/xlsxwriter/workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Se generó gráfico circular...
Se generó gráfico de barras...
Se generó un informe para el país Ecuador
Se finalizó la generación de informes.


/home/codespace/.python/current/lib/python3.10/site-packages/xlsxwriter/workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## 3. Envio Correo

In [9]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

def func_sender_email(**args):
    sender_email = args.get('sender_email')
    sender_password = args.get('sender_password')
    receiver_email = args.get('receiver_email')
    subject = args.get('subject')
    message = args.get('message')
    archivo_adjunto = args.get('archivo_adjunto')

    # Crear el objeto del mensaje
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))

    # Adjuntar el archivo
    with open(archivo_adjunto, "rb") as attachment:
        part = MIMEApplication(attachment.read(), Name=os.path.basename(archivo_adjunto))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(archivo_adjunto)}"'
        msg.attach(part)

    # Configurar el servidor SMTP
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # Iniciar sesión en el servidor SMTP
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)

        # Enviar el mensaje
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"Correo electrónico enviado a {receiver_email} con el informe adjunto.")

# Crear el directorio 'reportes' si no existe
os.makedirs('reportes', exist_ok=True)

# Cambiar al directorio de reportes
os.chdir('reportes')

# Llamar a la función func_sender_email con los parámetros proporcionados
func_sender_email(sender_email='dc.96.15.11@gmail.com',
                  sender_password='azzx igqs yovi gkgl',  # Reemplaza con la contraseña real
                  receiver_email='tatianagalarzah97@gmail.com',
                  subject='Reporte Brazil',
                  message='Reporte excel Brazil',
                  archivo_adjunto='Brazil.xlsx')



Correo electrónico enviado a tatianagalarzah97@gmail.com con el informe adjunto.
